In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import keras_models.dataGen as dg

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Activation, Conv2D, Concatenate, Dropout, MaxPool2D, Conv3D, Flatten, LeakyReLU, BatchNormalization

# importing in the preprocessed data

In [9]:
trainData = pkl.load(open("/home/sawerchessread/standardized_combined_simple_ensemble_train_data.pkl", 'rb'))

In [10]:
testData = pkl.load(open("/home/sawerchessread/standardized_combined_simple_ensemble_test_data.pkl", 'rb'))

In [11]:
validData = pkl.load(open("/home/sawerchessread/valid_standardized_combined_simple_ensemble_train_data.pkl", 'rb'))

In [ ]:
def generate_x_y(data):
    x_data = np.stack([datum[0] for datum in data])
    x_data = x_data.reshape((*x_data.shape, 1))
    x_data.transpose(0, 2, 1, 3)
    y_data = np.array([datum[1] for datum in data])
    y_data = keras.utils.to_categorical(y_data)
    return x_data, y_data

In [ ]:
testDataX, testDataY = generate_x_y(testData)
del testData

In [ ]:
trainDataX, trainDataY = generate_x_y(trainData)
del trainData

In [9]:
validDataX, validDataY = generate_x_y(validData)
del validData

# Making a quick set of architectures

In [12]:
dropout = 0.5

In [13]:
x = Input(((500, 21, 1)))
y1 = Conv2D(20, (3,3),  activation="relu",)(x)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = BatchNormalization()(y1)
y1 = Conv2D(40, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = BatchNormalization()(y1)
y1 = Conv2D(40, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = BatchNormalization()(y1)
y1 = Conv2D(40, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = BatchNormalization()(y1)
y1 = Conv2D(40, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = BatchNormalization()(y1)
y1 = Conv2D(40, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = Flatten()(y1)

W0728 04:52:50.831261 140037694965568 deprecation_wrapper.py:119] From /src/miniconda3/envs/py37/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 04:52:51.160865 140037694965568 deprecation_wrapper.py:119] From /src/miniconda3/envs/py37/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 04:52:51.380967 140037694965568 deprecation_wrapper.py:119] From /src/miniconda3/envs/py37/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 04:52:51.506352 140037694965568 deprecation_wrapper.py:119] From /src/miniconda3/envs/py37/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d inste

In [12]:
y0 = Conv2D(20, (2,2),  activation="relu",)(x)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = BatchNormalization()(y0)
y0 = Conv2D(40, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = BatchNormalization()(y0)
y0 = Conv2D(40, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = BatchNormalization()(y0)
y0 = Conv2D(40, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = BatchNormalization()(y0)
y0 = Conv2D(40, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = BatchNormalization()(y0)
y0 = Conv2D(40, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = Flatten()(y0)

In [13]:
y2 = Conv2D(20, (4,4),  activation="relu",)(x)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = BatchNormalization()(y2)
y2 = Conv2D(40, (4,4), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = BatchNormalization()(y2)
y2 = Conv2D(40, (4,4), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = BatchNormalization()(y2)
y2 = Conv2D(40, (4,4), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = BatchNormalization()(y2)
y2 = Conv2D(40, (4,4), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = BatchNormalization()(y2)
y2 = Conv2D(40, (4,4), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = Flatten()(y2)

In [14]:
y3 = Conv2D(20, (5,5),  activation="relu",)(x)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = BatchNormalization()(y3)
y3 = Conv2D(40, (5,5), activation="relu",)(y3)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = BatchNormalization()(y3)
y3 = Conv2D(40, (5,5), activation="relu",)(y3)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = BatchNormalization()(y3)
y3 = Conv2D(40, (5,5), activation="relu",)(y3)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = BatchNormalization()(y3)
y3 = Conv2D(40, (5,5), activation="relu",)(y3)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = Flatten()(y3)

In [14]:
# y = Concatenate()([y0, y1, y2, y3])

y = Dense(units=2, activation="softmax")(y1)
model = Model(inputs=x, outputs =y)

In [15]:
from keras.optimizers import Adam
adam = Adam(lr=0.00005)
model.compile(adam, loss="categorical_crossentropy", metrics=["acc"])

W0728 04:52:54.229012 140037694965568 deprecation_wrapper.py:119] From /src/miniconda3/envs/py37/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
model.summary()

In [26]:
type(model)

keras.engine.training.Model

In [27]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
cb = [EarlyStopping(patience=10, verbose=True), ModelCheckpoint("yolo.h5",save_best_only=True, verbose=True)]

In [20]:
72490*500*21*8

6089160000

In [20]:
sys.getsizeof(trainDataX)

144

In [31]:
trainDataX.shape

(72490, 500, 21, 1)

In [48]:
reload(dg)

<module 'keras_models.dataGen' from '/src/dbmi_eeg_clustering/keras_models/dataGen.py'>

In [30]:
trainData = dg.EdfDataGenerator(trainData, batch_size=32)

In [21]:
validData = dg.EdfDataGenerator(validData, batch_size=32)

In [31]:
trainData[0][0].shape

TypeError: can only concatenate list (not "int") to list

In [ ]:
history = model.fit_generator(trainData, epochs=1000, steps_per_epoch=32, callbacks=cb, validation_data=validData)

In [ ]:
model = keras.models.load_model("yolo.h5")

In [ ]:
y_pred = model.predict(testDataX)
roc_auc_score(testDataY.argmax(axis=1), y_pred.argmax(axis=1))

In [ ]:
dropout = 0.5

In [ ]:
x = Input(((500, 21, 1)))
y1 = Conv2D(50, (3,3),  activation="relu",)(x)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = Conv2D(200, (3,3), activation="relu",)(y1)
y1 = MaxPool2D(pool_size=(2, 1),)(y1)
y1 = Dropout(dropout)(y1)
y1 = Flatten()(y1)

In [ ]:
y0 = Conv2D(50, (2,2),  activation="relu",)(x)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = Conv2D(200, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = Conv2D(200, (2,2), activation="relu",)(y0)
y0 = MaxPool2D(pool_size=(2, 1),)(y0)
y0 = Dropout(dropout)(y0)
y0 = Flatten()(y0)

In [ ]:
y2 = Conv2D(50, (4,4),  activation="relu",)(x)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = Conv2D(200, (4,2), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = Conv2D(200, (4,2), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = Conv2D(200, (4,2), activation="relu",)(y2)
y2 = MaxPool2D(pool_size=(2, 1),)(y2)
y2 = Dropout(dropout)(y2)
y2 = Flatten()(y2)

In [ ]:
y3 = Conv2D(50, (5,5),  activation="relu",)(x)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = Conv2D(200, (5,5), activation="relu",)(y3)
y3 = MaxPool2D(pool_size=(2, 1),)(y3)
y3 = Dropout(dropout)(y3)
y3 = Flatten()(y3)

In [ ]:
y = Concatenate()([y0, y1, y2, y3])
y = Dense(units=2)(y)
model2 = Model(inputs=x, outputs =y)

In [ ]:
from keras.optimizers import Adam
adam = Adam(lr=0.002)
model2.compile(adam, loss="categorical_crossentropy", metrics=["binary_accuracy"])

In [ ]:
model2.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
cb = [EarlyStopping(patience=10), ModelCheckpoint("yolo.h5",save_best_only=True)]

In [ ]:
history = model2.fit(trainDataX, trainDataY, epochs=1000, callbacks=cb, validation_data=(validDataX, validDataY), )

In [ ]:
model2 = keras.models.load_model2("yolo.h5")

In [ ]:
y_pred = model2.predict(testDataX)
roc_auc_score(testDataY.argmax(axis=1), y_pred.argmax(axis=1))